In [1]:
import torchreid
model_name = 'resnet18'
dataset_name = 'soccernetv3'
datamanager = torchreid.data.ImageDataManager(
    root="reid-data",
    sources=dataset_name,
    targets=dataset_name,
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=["random_flip", "random_crop"]
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
SoccerNet valid set was already downloaded and unzipped at /home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/reid-data/soccernetv3/reid/valid.
SoccerNet train set was already downloaded and unzipped at /home/trewq/Desktop/Projects/RetailAnalytics/REID/RESEARCH/deep-person-reid/reid-data/soccernetv3/reid/train.
=> Loaded Soccernetv3
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    | 161443 |   248234 |      9189
  query    | 11638 |    11638 |      1751
  gallery  | 295

In [2]:
model = torchreid.models.build_model(
    name=model_name,
    num_classes=datamanager.num_train_pids, #2000 for Tango
    loss="softmax",
    pretrained=True
)
model = model.cuda()
optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

In [3]:
# from tools.visualize_actmap import visactmap
# test_loader = datamanager.test_loader
# visactmap(model, test_loader, f"log/visactmap_{model_name}_{dataset_name}", 128, 256, True, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

In [4]:
engine = torchreid.engine.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [6]:
engine.run(
    save_dir=f"log/{model_name}_{dataset_name}",
    max_epoch=60,
    eval_freq=5,
    print_freq=100,
    test_only=True,
    visrank=False,
    eval_metric='soccernetv3'
)

##### Evaluating soccernetv3 (source) #####
Extracting features from query set ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 117/117 [00:02<00:00, 54.26it/s]


Done, obtained 11638-by-512 matrix
Extracting features from gallery set ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 344/344 [00:10<00:00, 32.29it/s]


Done, obtained 34355-by-512 matrix
Speed: 0.0014 sec/batch
Computing distance matrix with metric=euclidean ...
Computing CMC and mAP ...


IndexError: index 242 is out of bounds for axis 0 with size 0